# Decision tree classifier

In this notebook, we will train a Decision tree classifier on a subset of the data. We will use grid-search to optimize the parameters.

In [1]:
import sys
sys.path.append('../scripts')

import numpy as np
import pandas as pd
import helpers_models as hm
from transforms import *
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV

First, load a sample of the data.

In [2]:
data = pd.read_hdf('../data/pivot_numbers_only.h5', 'data').sample(2000000)

In [3]:
first_week, last_week = data['tweets'].columns.min(), data['tweets'].columns.max()

In [4]:
data = hm.make_target(data, last_week)

In [5]:
data.shape

(2000000, 53)

In [6]:
first_week

23

In [7]:
last_week

36

Split the data into train and test.

In [8]:
train, test = train_test_split(data)

Now make a pipeline for transforming and predicting.

In [9]:
pipeline = Pipeline([
    ('limiter', WeeksLimiter(first_week=25, target_week=last_week)),
    ('normal', Normalizer()),
    ('decay', TimeDecayApplier(target_week=last_week)),
    ('tree', DecisionTreeClassifier())
])

Now let's check the params of the pipeline for grid-search to optimize.

In [10]:
pipeline.get_params()

{'decay': TimeDecayApplier(skip=False, target_week=36),
 'decay__skip': False,
 'decay__target_week': 36,
 'limiter': WeeksLimiter(first_week=25, target_week=36),
 'limiter__first_week': 25,
 'limiter__target_week': 36,
 'normal': Normalizer(skip=False),
 'normal__skip': False,
 'steps': [('limiter', WeeksLimiter(first_week=25, target_week=36)),
  ('normal', Normalizer(skip=False)),
  ('decay', TimeDecayApplier(skip=False, target_week=36)),
  ('tree',
   DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
               max_features=None, max_leaf_nodes=None,
               min_impurity_split=1e-07, min_samples_leaf=1,
               min_samples_split=2, min_weight_fraction_leaf=0.0,
               presort=False, random_state=None, splitter='best'))],
 'tree': DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
             max_features=None, max_leaf_nodes=None,
             min_impurity_split=1e-07, min_samples_leaf=1,
             min_

In [11]:
params = {
    'limiter__first_week': [28, 30, 32],
    'normal__skip': [False, True],
    'decay__skip': [False, True],
    'tree__max_depth': [3, 6]
}

In [12]:
model = GridSearchCV(pipeline, params, n_jobs=-1, pre_dispatch='2*n_jobs', verbose=2, error_score=0)

The next step is to train the model and optimize the params.

In [13]:
%%time
model.fit(train.drop('target', axis=1, level=0), train['target'])

Fitting 3 folds for each of 24 candidates, totalling 72 fits
[CV] decay__skip=False, limiter__first_week=28, normal__skip=False, tree__max_depth=3 
[CV] decay__skip=False, limiter__first_week=28, normal__skip=False, tree__max_depth=3 
[CV] decay__skip=False, limiter__first_week=28, normal__skip=False, tree__max_depth=3 
[CV] decay__skip=False, limiter__first_week=28, normal__skip=False, tree__max_depth=6 
[CV] decay__skip=False, limiter__first_week=28, normal__skip=False, tree__max_depth=6 
[CV]  decay__skip=False, limiter__first_week=28, normal__skip=False, tree__max_depth=3, total=  10.4s
[CV] decay__skip=False, limiter__first_week=28, normal__skip=False, tree__max_depth=6 
[CV]  decay__skip=False, limiter__first_week=28, normal__skip=False, tree__max_depth=3, total=  12.2s
[CV] decay__skip=False, limiter__first_week=28, normal__skip=True, tree__max_depth=3 
[CV]  decay__skip=False, limiter__first_week=28, normal__skip=False, tree__max_depth=3, total=  15.2s
[CV] decay__skip=False, l

[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:  3.7min


[CV] decay__skip=False, limiter__first_week=32, normal__skip=False, tree__max_depth=6 
[CV] decay__skip=False, limiter__first_week=32, normal__skip=True, tree__max_depth=3 
[CV]  decay__skip=False, limiter__first_week=30, normal__skip=True, tree__max_depth=6, total=  48.7s
[CV]  decay__skip=False, limiter__first_week=32, normal__skip=False, tree__max_depth=3, total=  21.1s
[CV] decay__skip=False, limiter__first_week=32, normal__skip=True, tree__max_depth=3 
[CV] decay__skip=False, limiter__first_week=32, normal__skip=True, tree__max_depth=3 
[CV]  decay__skip=False, limiter__first_week=32, normal__skip=False, tree__max_depth=6, total=  19.8s
[CV] decay__skip=False, limiter__first_week=32, normal__skip=True, tree__max_depth=6 
[CV]  decay__skip=False, limiter__first_week=32, normal__skip=False, tree__max_depth=6, total=  16.6s
[CV] decay__skip=False, limiter__first_week=32, normal__skip=True, tree__max_depth=6 
[CV]  decay__skip=False, limiter__first_week=32, normal__skip=False, tree__m

[CV]  decay__skip=True, limiter__first_week=32, normal__skip=True, tree__max_depth=6, total=  11.4s
[CV]  decay__skip=True, limiter__first_week=32, normal__skip=True, tree__max_depth=6, total=  10.3s


[Parallel(n_jobs=-1)]: Done  72 out of  72 | elapsed:  7.4min finished


CPU times: user 4min 8s, sys: 36.1 s, total: 4min 44s
Wall time: 7min 45s


GridSearchCV(cv=None, error_score=0,
       estimator=Pipeline(steps=[('limiter', WeeksLimiter(first_week=25, target_week=36)), ('normal', Normalizer(skip=False)), ('decay', TimeDecayApplier(skip=False, target_week=36)), ('tree', DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            presort=False, random_state=None, splitter='best'))]),
       fit_params={}, iid=True, n_jobs=-1,
       param_grid={'limiter__first_week': [28, 30, 32], 'normal__skip': [False, True], 'decay__skip': [False, True], 'tree__max_depth': [3, 6]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring=None, verbose=2)

Finally, use the model to predict test values and scores. We will use the test data for predictions.

In [14]:
%%time
predicted = model.predict(test.drop('target', axis=1, level=0))
report = classification_report(test['target'], predicted, digits=5)

CPU times: user 3.2 s, sys: 2.37 s, total: 5.58 s
Wall time: 5.51 s


In [15]:
print(report)

             precision    recall  f1-score   support

      False    0.91819   0.98218   0.94911    442005
       True    0.71036   0.33306   0.45350     57995

avg / total    0.89409   0.90689   0.89162    500000



Also check the best parameters grid-search found.

In [16]:
model.best_params_

{'decay__skip': False,
 'limiter__first_week': 28,
 'normal__skip': True,
 'tree__max_depth': 6}